In [ ]:
def model(X_train, y_train, params, layers, decay, interval=1000,learning_rate=0.01, decay_rate=0.3, lambda_value=0, batch_size=64,
         epoch=100, loops=100, beta1=0.9, beta2=0.999, epsilon=1e-8, threshold=0.5, mini = True, adam_optim = True, testing = False):

    m=X_train.shape[0]

    all_costs_test = []
    all_accs_test = []
    
    all_costs = []
    all_accs = []
    
    print(f'Model -> hidden layers: {len(layers)-2} | minibatch: {str(mini)} | lambda: {lambda_value} | adam: {str(adam_optim)} | decay: {decay} | lr: {learning_rate}')

    if adam_optim == True:
        t = 0
        v,s = adam_init(params)


    # MINIBATCHE
    if mini == True:

        seed = 0

        iterations = math.ceil(m / batch_size) 

        for i in tqdm(range(epoch), desc='epoch', colour='black'):

            
            seed += 1
            
            all_batches = batch_maker(X_train, y_train, batch_size, seed)

            for j in range(iterations):
                
                X_batch, y_batch = all_batches[j]

                AL, cache, _ = forward_prop(X_batch, y_batch, params, lambda_value)
                
                grads = back_prop(AL, X_batch, y_batch, layers, cache, lambda_value)

                if adam_optim == True:
                    t+=1
                    new_params = update_params_adam(params, grads, v, s, t, learning_rate = learning_rate, beta1=beta1, beta2=beta2, epsilon=epsilon)
                    params = new_params.copy()

                else:
                    new_params = update_params(params, grads, learning_rate = learning_rate)
                    params = new_params.copy()


            if decay == 'exp':
                
                new_lr = update_lr(learning_rate, i, decay_rate)
                learning_rate = new_lr

            elif (decay == 'time') & (i%interval==0):
                
                new_lr = update_lr_time(learning_rate, i, decay_rate, interval)
                learning_rate = new_lr
                
            else:
                pass
            
            _, accuracy, cost = predict(X_train, y_train, params, lambda_value = lambda_value, threshold = threshold)
            
            all_costs.append(cost) 
            all_accs.append(accuracy)

            if testing == True:

                _, acc_test, cost_test = predict(X_test, y_test, params, lambda_value, threshold=threshold)

                all_costs_test.append((cost, cost_test))
                all_accs_test.append((accuracy, acc_test))
                
        return all_costs, all_accs, params, AL, all_costs_test, all_accs_test

    else: # BATCH 

        for i in tqdm(range(epoch), desc='iteration', colour='black'):

            AL, cache, _ = forward_prop(X_train, y_train, params, lambda_value)

            _, acc, cost = predict(X_train, y_train, params, lambda_value, threshold=threshold)
            
            grads = back_prop(AL, X_train, y_train, layers, cache, lambda_value)
            
            all_costs.append(cost)
            all_accs.append(acc)
            
            if testing == True:
                _, acc_test, cost_test = predict(X_test, y_test, params, lambda_value, threshold=threshold)            
                
                
                all_costs_test.append((cost, cost_test))
                all_accs_test.append((acc, acc_test))
            
            if adam_optim == True:
                t += 1
                new_params = update_params_adam(params, grads, v, s, t, learning_rate = learning_rate, beta1 = beta1, beta2 = beta2, epsilon = epsilon)
                params = new_params.copy()

            else:
                new_params = update_params(params, grads, learning_rate = learning_rate)
                params = new_params.copy()

            if decay == 'exp':

                new_lr = update_lr_exp(learning_rate, i, decay_rate)
                learning_rate = new_lr

            elif (decay == 'time') & (i % interval==0):

                new_lr = update_lr_time(learning_rate, i, decay_rate, interval)
                learning_rate = new_lr
                
            else:
                pass
        
        return all_costs, all_accs, params, AL, all_costs_test, all_accs_test